# Lending Club Loan Prediction

In this notebook we will explore loan data from Lending Club between 2007 and 2019 to predict whether loan applicants will default on a loan or not.

The original data set can be found here: https://www.kaggle.com/wendykan/lending-club-loan-data

## Exploratory Data Analysis

We will begin by exploring this data set that was posted by Lending Club.

In [13]:
# Import libraries for initial data analysis

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Display plots in the notebook inline
%matplotlib inline

# Import our dataset
loans = pd.read_csv('./data/loan.csv', low_memory=False)

# View all columns
pd.set_option('display.max_columns', None)

#### Gather basic information

In [3]:
# A whopping 2.26M records with 145 features
loans.shape

(2260668, 145)

In [4]:
loans.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260668 entries, 0 to 2260667
Data columns (total 145 columns):
id                                            float64
member_id                                     float64
loan_amnt                                     int64
funded_amnt                                   int64
funded_amnt_inv                               float64
term                                          object
int_rate                                      float64
installment                                   float64
grade                                         object
sub_grade                                     object
emp_title                                     object
emp_length                                    object
home_ownership                                object
annual_inc                                    float64
verification_status                           object
issue_d                                       object
loan_status                       

## Feature Engineering & Predictive Model

The aim of this section is to create a model which can reliably predict the outcome of an issued loan based on the given features. This is a categorical model as the output will either be 1 denoting a good loan or 0 denoting a bad loan. In order to build a predictive model we will first have to do some feature engineering to prep our data set.

### 1.) Clean up features & records

There are a lot of features included with this data set, each requiring some cleanup. To prevent this from overwhelming, we'll just focus on a key set of features for now.

### 1.) Drop unnecessary records

*Keep only key features* - There are a few feature columns such as member_id which will be irrelevant to our predictive model.

*Remove records prior to 2013* - Lending Club was still in growth mode between 2007-2015 (number of loans issued was doubling year over year) and their underwriting process was likely different than it is today. However we want to include loans which have reached full maturity (3 and 5 year terms) which will require us to include data as far back as 2013.

*Keep only Fully Paid and Charged off* - Remove records with loan_status other than either Fully Paid (good loan) or Charged Off (bad loan) to simplify prediction to a binary output. This will also include removing current loans since we have no way of predicting the outcome of current loan that has yet to come to term or be paid off.

(NOT COMPLETE) *Remove redundant features* - There are features that are redundant and will have no impact on the outcome of the loan. We will drop these features.

In [5]:
# Let's take a look at some of the entries
loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,Chef,10+ years,RENT,55000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,109xx,NY,18.24,0.0,Apr-2001,1.0,NaN,45.0,9.0,1.0,4341,10.3,34.0,w,2386.02,2386.02,167.02,167.02,113.98,53.04,0.0,0.0,0.0,Feb-2019,84.92,Mar-2019,Feb-2019,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0,0.0,16901.0,2.0,2.0,1.0,2.0,2.0,12560.0,69.0,2.0,7.0,2137.0,28.0,42000.0,1.0,11.0,2.0,9.0,1878.0,34360.0,5.9,0.0,0.0,140.0,212.0,1.0,1.0,0.0,1.0,NaN,2.0,NaN,0.0,2.0,5.0,3.0,3.0,16.0,7.0,18.0,5.0,9.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,60124.0,16901.0,36500.0,18124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,Postmaster,10+ years,MORTGAGE,90000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,713xx,LA,26.52,0.0,Jun-1987,0.0,71.0,75.0,13.0,1.0,12315,24.2,44.0,w,29387.75,29387.75,1507.11,1507.11,612.25,894.86,0.0,0.0,0.0,Feb-2019,777.23,Mar-2019,Feb-2019,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0,1208.0,321915.0,4.0,4.0,2.0,3.0,3.0,87153.0,88.0,4.0,5.0,998.0,57.0,50800.0,2.0,15.0,2.0,10.0,24763.0,13761.0,8.3,0.0,0.0,163.0,378.0,4.0,3.0,3.0,4.0,NaN,4.0,NaN,0.0,2.0,4.0,4.0,9.0,27.0,8.0,14.0,4.0,13.0,0.0,0.0,0.0,6.0,95.0,0.0,1.0,0.0,372872.0,99468.0,15000.0,94072.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,Administrative,6 years,MORTGAGE,59280.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt cons

In [14]:
# Let's convert the issue_d column to actual date values that we can work with and create two new columns
loans['issue_year'] = pd.to_datetime(loans['issue_d']).dt.year
loans['issue_month'] = pd.to_datetime(loans['issue_d']).dt.month

In [15]:
####### Remove older records #######

# Remove records prior to 2013
loans = loans[loans.issue_year >= 2013]

####### Simplify loan status #######

# Keep only records if loans status is Fully Paid or Charged Off
loans = loans.loc[loans.loan_status.isin(['Fully Paid', 'Charged Off'])]

####### Reduce number of features #######

key_features = [
    # Loan
    'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'issue_year', 'purpose', 'out_prncp',
    'application_type', 'total_rec_int', 'total_pymnt',
    
    # Applicant
    'emp_length', 'home_ownership', 'annual_inc', 'annual_inc_joint',
    
    # Basic credit history
    'open_acc', 'total_acc', 'open_rv_24m', 'inq_fi', 'acc_open_past_24mths', 'avg_cur_bal', 
    'delinq_amnt', 'mort_acc', 'num_bc_sats', 'num_bc_tl', 'num_sats', 
    
    # Bad credit history
    'collections_12_mths_ex_med', 'pub_rec', 'delinq_2yrs', 'acc_now_delinq', 'mths_since_last_major_derog', 
    'mths_since_last_delinq', 'tot_coll_amt', 'mths_since_recent_bc_dlq', 'num_tl_120dpd_2m', 'num_tl_30dpd', 
    'num_tl_90g_dpd_24m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'tax_liens', 
    
    # Fees
    'total_rec_late_fee',
    
    # Target
    'loan_status'
]

# Keep only key features
loans = loans[key_features]

loans.head()

,loan_amnt,term,int_rate,installment,grade,issue_year,purpose,out_prncp,application_type,total_rec_int,total_pymnt,emp_length,home_ownership,annual_inc,annual_inc_joint,open_acc,total_acc,open_rv_24m,inq_fi,acc_open_past_24mths,avg_cur_bal,delinq_amnt,mort_acc,num_bc_sats,num_bc_tl,num_sats,collections_12_mths_ex_med,pub_rec,delinq_2yrs,acc_now_delinq,mths_since_last_major_derog,mths_since_last_delinq,tot_coll_amt,mths_since_recent_bc_dlq,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,pct_tl_nvr_dlq,percent_bc_gt_75,tax_liens,total_rec_late_fee,loan_status
100,30000,36 months,22.35,1151.16,D,2018,debt_consolidation,0.0,Joint App,26.44,30026.442500,5 years,MORTGAGE,100000.0,150000.0,11.0,19.0,1.0,1.0,4.0,42939.0,0.0,1.0,3.0,5.0,11.0,0.0,1.0,0.0,0.0,NaN,51.0,0.0,NaN,0.0,0.0,0.0,89.5,33.3,0.0,0.0,Fully Paid
152,40000,60 months,16.14,975.71,C,2018,credit_card,0.0,Joint App,856.68,40856.678960,< 1 year,MORTGAGE,45000.0,92000.0,18.0,37.0,1.0,4.0,5.0,15059.0,0.0,1.0,7.0,7.0,18.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,100.0,42.9,0.0,0.0,Fully Paid
170,20000,36 months,7.56,622.68,A,2018,credit_card,0.0,Joint App,215.79,20215.792430,10+ years,MORTGAGE,100000.0,190000.0,9.0,19.0,2.0,2.0,5.0,57309.0,0.0,5.0,5.0,6.0,9.0,0.0,0.0,0.0,0.0,NaN,48.0,0.0,NaN,0.0,0.0,0.0,94.7,20.0,0.0,0.0,Fully Paid
186,4500,36 months,11.31,147.99,B,2018,credit_card,0.0,Individual,49.22,4549.217149,10+ years,RENT,38500.0,NaN,12.0,25.0,7.0,0.0,8.0,2428.0,0.0,0.0,5.0,6.0,12.0,0.0,0.0,0.0,0.0,25.0,25.0,306.0,NaN,0.0,0.0,0.0,91.7,0.0,0.0,0.0,Fully Paid
215,8425,36 months,27.27,345.18,E,2018,credit_card,0.0,Joint App,310.15,8735.148975,3 years,MORTGAGE,450000.0,515000.0,21.0,37.0,4.0,7.0,10.0,34516.0,0.0,4.0,8.0,10.0,21.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,100.0,50.0,0.0,0.0,Fully Paid


### 2.) Eliminate class imbalance

After examining the number of records remaining, we can see that the data set has imbalanced classes where only 20% of the records show a bad loan. Since we still have 247,341 bad loans to work with, we will down-sample our good loans to match our number of bad loans.

For future iterations of this project, we may consider using tree based algorithms which are more capable of handling imbalanced data sets.

In [17]:
# Import resampling module for sklearn
from sklearn.utils import resample

# Separate good vs bad loans
good_loans = loans[loans.loan_status == 'Fully Paid']
bad_loans = loans[loans.loan_status == 'Charged Off']
 
# Downsample good loans
good_loans_downsampled = resample(good_loans, replace=False, n_samples=247341, random_state=123)

# Recombine loans
loans = pd.concat([good_loans_downsampled, bad_loans])

In [18]:
# Confirm that are classes are now balanced
loans["loan_status"].value_counts()

Fully Paid     247341
Charged Off    247341
Name: loan_status, dtype: int64

### 3.) Clean up remaining data

### 3.1.) Deal with categorical features

There are many categorical features which will need to be encoded to numeric values before we can build our predictive model.

In [56]:
# Taking a look at the categorical features

loan_objects = loans.select_dtypes(include=['object']).copy()
loan_objects.head()

,term,grade,purpose,application_type,home_ownership,loan_status
1569316,60 months,C,credit_card,Individual,RENT,Fully Paid
1692896,60 months,E,debt_consolidation,Individual,MORTGAGE,Fully Paid
1159712,36 months,B,debt_consolidation,Individual,OWN,Fully Paid
1098268,36 months,A,other,Individual,RENT,Fully Paid
1375762,60 months,B,credit_card,Individual,RENT,Fully Paid


In [57]:
loan_objects[loan_objects.isnull().any(axis=1)]

,term,grade,purpose,application_type,home_ownership,loan_status


We can not only see that employment length has some empty values, but also that this feature could be converted to numeric instead of categorical values.

In [39]:
# Convert employment length feature to numeric values

def convert_employment_length(x):
    if x == '< 1 year':
        return 0.5
    if x == '1 year':
        return 1
    if x == '2 years':
        return 2
    if x == '3 years':
        return 3
    if x == '4 years':
        return 4
    if x == '5 years':
        return 5
    if x == '6 years':
        return 6
    if x == '7 years':
        return 7
    if x == '8 years':
        return 8
    if x == '9 years':
        return 9
    if x == '10+ years':
        return 10
    else:
        return 0

loans.emp_length = loans.emp_length.apply(convert_employment_length)

In [ ]:
# Perform one hot encoding on remaining categorical values
loans = pd.get_dummies(loans, columns=['term', 'grade', 'purpose', 'application_type', 'home_ownership', 'loan_status'])

In [75]:
#loans.drop(labels='loan_status_Charged Off', axis=1)
loans = loans.drop(labels='loan_status_Charged Off', axis=1)

In [73]:
loans = loans.rename(columns={'loan_status_Fully Paid': 'loan_status'})

### 3.2.) Deal with numeric features

*Replace NaN values* - Numeric features such as mths_since_last_delinq will include a NaN value if the loan applicant does not meet that given criteria. We will want to convert these values to 0.

*Scale numeric features* - We have a diverse scale of our numeric features. Scale these accordingly.

### 5.) Train test split

Split data set into training and test data.

### 6.) Test multiple models

Try a few different models to determine which will most reliably predict loan status.

## Which features determine a bad loan

Here we will want to determine which features are most influential in causing a loan to be 'bad'.

### Prediction target
Loan status (loan_status) is an appropriate target variable from this data since we are attempting to predict if a customer will pay back a loan or not.

In [8]:
loans["loan_status"].value_counts()

Fully Paid                                             1041952
Current                                                 919695
Charged Off                                             261655
Late (31-120 days)                                       21897
In Grace Period                                           8952
Late (16-30 days)                                         3737
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     31
Name: loan_status, dtype: int64

### Importance of Issue Date
Issue date on these loans is important due to the fact that more recent loans may not have fully come to term yet. This will cause the number of 'bad' loans to appear disproportionately low.

Additionally, Lending Club was founded in 2006 which accounts for the relatively small number of loans which were issued prior to 2010.

In [10]:
loans.issue_year.value_counts()

2018    495242
2017    443579
2016    434407
2015    421095
2014    235629
2013    134814
2012     53367
2011     21721
2010     12537
2009      5281
2008      2393
2007       603
Name: issue_year, dtype: int64

In [11]:
# When we take a look at the term of issued loans, it looks like LC either issues 3 or 5 year loans.
loans.term.value_counts()

 36 months    1609754
 60 months     650914
Name: term, dtype: int64

In [12]:
# Loans prior to 2013 are "mostly" either Paid Off (Good Loan) or Charged Off (Bad Loan)
loans[loans.issue_year == 2013].loan_status.value_counts()

Fully Paid            113771
Charged Off            21022
Current                    9
Late (31-120 days)         8
Late (16-30 days)          2
In Grace Period            2
Name: loan_status, dtype: int64

## Manual Review of features

Important features we will want to dig into could be:

#### Term of loan
LC issues 3 or 5 year loans. We should investigate if one or the other has a higher likelihood of bad loans.

#### Interest Rate and Grade
Interest rate is likely correlated with the grade of the loan as lower grade loans typically have higher interest rates. We will want to explore if low grade (high interest) loans have a higher likelihood of being a bad loan.

#### Annual Income and Verification Status
We should investigate if annual income has an impact on bad loans. We may also want to investigate if loans which have annual incomes which could not be verified are more likely to be a bad loan.

#### Purpose
We may want to look at the type of loan (Debt Consolidation, Car, Credit Card, etc) has an impact on bad loans.

#### Delinquency in last 2 years
This seems like it should clearly impact likelihood of a bad loan

#### Public records
I'm very curious in public records which may be an indication of a bad loan. These would include features: pub_rec, pub_rec_bankruptcies, and tax_liens.

#### Credit behavior
Credit behavior should be a heavy influencer of good/bad loans and there are a lot of features relating to this which pertain to current balance on accounts, revolving bankcards, credit lines, mortgage accounts, credit limit, etc.

In [13]:
# Step 1
# Remove records prior to 2015 as the company was still in hyper growth mode (number of loans issued was doubling year
# over year) and their underwriting process was likely different than it is today.

# Step 2
# Remove records with loan_status other than either Fully Paid (good loan) or Charged Off (bad loan) to simplify
# prediction a binary output

# Step 3
# Create an equal distribution of records for good and bad loans

# Step 4
# Run a correlation analysis to determine if any features are correlated.

# Step 5
# Carve out a training and test set from the data

# Step 6
# This is a classification problem. So consider if regression modeling is appropriate...
# Build and run a linear regression model to determine predictive accuracy
# Build and run a KNN classification model to determine predictive accuracy
# Build and run a logistic regression model to determine predictive accuracy
# Try and build a neural network to determine predictive accuracy


In [14]:
# Step 1
# Remove records prior to 2015 as the company was still in hyper growth mode (number of loans issued was doubling year
# over year) and their underwriting process was likely different than it is today.

